In [1]:
import numpy as np

In [2]:
# Load the data
data = np.loadtxt("databp.csv", delimiter=',', skiprows=0)  # skiprows=0 to not skip the header
print(data)

[[0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [2. 3.]
 [0. 2.]
 [2. 3.]
 [1. 1.]
 [0. 2.]
 [2. 1.]
 [2. 1.]
 [3. 2.]
 [1. 1.]
 [2. 3.]
 [1. 3.]
 [2. 3.]
 [1. 6.]
 [1. 3.]
 [0. 6.]
 [0. 4.]
 [1. 1.]
 [1. 3.]
 [2. 2.]
 [2. 3.]
 [1. 2.]
 [3. 3.]
 [0. 0.]
 [1. 2.]
 [2. 1.]
 [2. 1.]
 [1. 0.]
 [1. 1.]
 [1. 3.]
 [0. 4.]
 [0. 2.]
 [1. 2.]
 [1. 0.]
 [1. 1.]
 [1. 2.]
 [2. 3.]
 [0. 1.]
 [2. 2.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [0. 0.]
 [1. 2.]
 [4. 3.]
 [3. 4.]
 [4. 3.]
 [4. 4.]
 [6. 4.]
 [4. 1.]
 [1. 1.]
 [2. 2.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 2.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [2. 3.]
 [2. 2.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 2.]
 [0. 1.]
 [1. 3.]
 [2. 0.]
 [1. 0.]
 [1. 3.]
 [0. 1.]
 [0. 1.]
 [0. 4.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [1. 2.]
 [3. 2.]
 [2. 6.]
 [7. 5.]
 [6. 5.]
 [4. 1.]
 [2. 2.]
 [1. 7.]
 [0. 1.]
 [0. 5.]
 [0. 5.]
 [2. 2.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 1.]
 [1. 0.]
 [2. 0.]
 [5. 0.]
 [3. 1.]
 [4. 1.]
 

In [3]:
# Center the data
def center_data(data):
    means = data.mean(axis=0)
    Y = data - means
    return Y

In [4]:
centered_data = center_data(data)
print(centered_data)

[[-1.11  -1.784]
 [-1.11  -1.784]
 [-1.11  -0.784]
 [-1.11  -0.784]
 [-0.11  -1.784]
 [ 0.89   1.216]
 [-1.11   0.216]
 [ 0.89   1.216]
 [-0.11  -0.784]
 [-1.11   0.216]
 [ 0.89  -0.784]
 [ 0.89  -0.784]
 [ 1.89   0.216]
 [-0.11  -0.784]
 [ 0.89   1.216]
 [-0.11   1.216]
 [ 0.89   1.216]
 [-0.11   4.216]
 [-0.11   1.216]
 [-1.11   4.216]
 [-1.11   2.216]
 [-0.11  -0.784]
 [-0.11   1.216]
 [ 0.89   0.216]
 [ 0.89   1.216]
 [-0.11   0.216]
 [ 1.89   1.216]
 [-1.11  -1.784]
 [-0.11   0.216]
 [ 0.89  -0.784]
 [ 0.89  -0.784]
 [-0.11  -1.784]
 [-0.11  -0.784]
 [-0.11   1.216]
 [-1.11   2.216]
 [-1.11   0.216]
 [-0.11   0.216]
 [-0.11  -1.784]
 [-0.11  -0.784]
 [-0.11   0.216]
 [ 0.89   1.216]
 [-1.11  -0.784]
 [ 0.89   0.216]
 [-1.11  -1.784]
 [-1.11  -0.784]
 [-0.11  -1.784]
 [-1.11  -1.784]
 [-0.11   0.216]
 [ 2.89   1.216]
 [ 1.89   2.216]
 [ 2.89   1.216]
 [ 2.89   2.216]
 [ 4.89   2.216]
 [ 2.89  -0.784]
 [-0.11  -0.784]
 [ 0.89   0.216]
 [-0.11  -1.784]
 [-1.11  -0.784]
 [-1.11  -0.78

In [5]:
# Fit VAR model
def fit_VAR(data, p):
    n, k = data.shape
    X = data
    
    # Construct the regressor matrix Xprime
    Xprime = np.zeros((n - p, k * p))
    X_target = np.zeros((n - p, k))
    
    for t in range(p, n):
        Xprime_row = []
        for lag in range(1, p + 1):
            Xprime_row.extend(X[t - lag])
        Xprime[t - p, :] = Xprime_row
        X_target[t - p] = X[t]
    
    # Estimate coefficients using OLS
    beta = np.linalg.inv(Xprime.T @ Xprime) @ Xprime.T @ X_target
    # Predicting X_hat
    X_hat = Xprime @ beta
    residuals = X[p:] - X_hat
    
    return residuals, beta

In [6]:
residuals, beta = fit_VAR(centered_data, 3)
print(beta)

# changement de forme pour adapter et comparer a l'output de la fonction de fit statsmodels
beta_reshaped = beta.T.reshape(2, 3, 2)
print(beta_reshaped)

[[ 0.52758547 -0.04720584]
 [ 0.07779866  0.32444785]
 [ 0.13733772  0.09065081]
 [ 0.01198667  0.07576308]
 [-0.03262743 -0.0832963 ]
 [-0.03551175  0.07187789]]
[[[ 0.52758547  0.07779866]
  [ 0.13733772  0.01198667]
  [-0.03262743 -0.03551175]]

 [[-0.04720584  0.32444785]
  [ 0.09065081  0.07576308]
  [-0.0832963   0.07187789]]]


In [8]:
x1 = np.random.rand(5,1)
print(x1)
x2 = np.random.rand(5,1)
print(x2)
xc = np.concatenate((x1, x2), axis = 1)
print(xc, np.shape(xc))
xc[3,:] = xc[1,0], xc[2,0]
print(xc[3:,0])


[[0.32514028]
 [0.59745533]
 [0.23169089]
 [0.0109393 ]
 [0.51107997]]
[[0.20949875]
 [0.26208052]
 [0.51979602]
 [0.64102952]
 [0.57917436]]
[[0.32514028 0.20949875]
 [0.59745533 0.26208052]
 [0.23169089 0.51979602]
 [0.0109393  0.64102952]
 [0.51107997 0.57917436]] (5, 2)
[0.59745533 0.51107997]


In [23]:
n = 497
print(data)
X1 = data[p+1:, 0]
X2 = data[p:, 1]
#Xs = np.zeros((n,2))
#Xs[:,0] = X1
#Xs[:,1] = X2
#print(f"xs:{Xs}")
print(np.shape(X1))

[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 2.]
 [1. 0.]
 [3. 2.]
 [0. 0.]
 [3. 1.]
 [1. 1.]
 [1. 1.]
 [3. 0.]
 [3. 1.]
 [3. 0.]
 [1. 0.]
 [0. 0.]
 [0. 2.]
 [2. 2.]
 [3. 2.]
 [2. 0.]
 [2. 1.]
 [0. 1.]
 [1. 2.]
 [1. 3.]
 [0. 4.]
 [0. 3.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [2. 1.]
 [3. 2.]
 [3. 2.]
 [3. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [2. 1.]
 [2. 1.]
 [0. 2.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [0. 3.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [0. 2.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 1.]
 [0. 2.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [2. 1.]
 [2. 1.]
 [2. 3.]
 [3. 0.]
 [3. 1.]
 [0. 0.]
 [1. 1.]
 [2. 1.]
 [4. 1.]
 [4. 1.]
 [1. 2.]
 [3. 3.]
 [1. 2.]
 [0. 1.]
 [0. 2.]
 [1. 3.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [2. 1.]
 [3. 1.]
 [0. 1.]
 [0. 4.]
 [1. 1.]
 [0. 5.]
 [0. 3.]
 [0. 2.]
 [0. 1.]
 [1. 0.]
 [2. 1.]
 [0. 1.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 

In [7]:
print(residuals)

[[-3.89126306e-01 -3.10476364e-01]
 [ 5.98887021e-01 -1.38623945e+00]
 [ 1.18461198e+00  1.91353635e+00]
 [-1.70172053e+00 -2.74890844e-02]
 [ 1.25506701e+00  1.03978122e+00]
 [-4.52077792e-01 -1.06552762e+00]
 [-1.15632371e+00  1.84382822e-01]
 [ 1.55554076e+00 -8.50379175e-01]
 [ 5.99868600e-01 -3.56172446e-01]
 [ 1.34006417e+00  3.83114827e-01]
 [-1.23557686e+00 -6.55656693e-01]
 [ 7.48068342e-01  1.40796559e+00]
 [-5.80313165e-01  1.07475884e+00]
 [ 6.85194632e-01  6.90661317e-01]
 [-6.01402181e-01  3.76805848e+00]
 [-4.77177851e-01 -4.26438006e-01]
 [-1.10977673e+00  3.49356340e+00]
 [-7.05719405e-01  4.01373296e-01]
 [ 4.44720359e-01 -1.87073578e+00]
 [ 2.48412069e-01  1.00240980e+00]
 [ 9.20413530e-01 -3.66091676e-01]
 [ 3.72745547e-01  1.15296581e+00]
 [-7.59380652e-01 -3.30125542e-01]
 [ 1.83113248e+00  1.02652758e+00]
 [-2.11700097e+00 -2.10897259e+00]
 [ 3.44350128e-01  4.54270322e-01]
 [ 1.20990711e+00 -5.53463144e-01]
 [ 3.94391703e-01 -4.58241666e-01]
 [-6.27308406e-01 -1

In [8]:
# Function to estimate VAR-ARMA parameters
def estimate_VAR_ARMA(data, p, residuals): #p = ordre du VAR sur les donnees centrees, necessaire pour les retards et valeurs manquantes des residus
    T, N = data.shape
    assert N == 2, "This implementation is for bivariate VAR only."
    print(f"T = {T}")
    n = T - p  # Number of rows after adjusting for lags
    Xprime = np.zeros((n-1, 5)) #n lignes pour les pbs, 3 = 1 cst + 2 termes retard X_t-1 + 2 terme retard et-1
    Xprime[:, 0] = 1  # colonne de la constante 
    print(f"n = {n}")
    for t in range(0, n-1):
        Xprime[t, 1:3] = data[p+t, :] #pour t= 0 le terme le plus lointain dans nos données (celui de retard 1) est le (p+1)eme element de nos données, et le premier qui a un terme de retard correspondant
        Xprime[t, 3:5] = residuals[t, :]
    print(Xprime)
    X1 = data[p+1:, 0]
    X2 = data[p+1:, 1]
    Xs = np.zeros((n-1,2))
    Xs[:,0] = X1
    Xs[:,1] = X2

    # Estimate coefficients using OLS
    beta = np.linalg.inv(Xprime.T @ Xprime) @ Xprime.T @ Xs

    Phi = np.zeros((2, 2))
    Theta = np.zeros((2, 2))
    omega = [beta[0, 0], beta[0, 1]]
    Theta[0, :] = beta[3,0], beta[4,0]
    Theta[1, :] = beta[3,1], beta[4,1]
    Phi[0, :] = beta[1,0], beta[1,1]
    Phi[1, :] = beta[2,0], beta[2,1]
    
    return omega, Theta, Phi

In [9]:
# Assuming `data` and `residuals` are already defined
p = 3
omega, Theta, Phi = estimate_VAR_ARMA(data, p, residuals)
print("Omega:\n", omega)
print("Theta:\n", Theta)
print("Phi:\n", Phi)

T = 500
n = 497
[[ 1.          0.          1.         -0.38912631 -0.31047636]
 [ 1.          1.          0.          0.59888702 -1.38623945]
 [ 1.          2.          3.          1.18461198  1.91353635]
 ...
 [ 1.          0.          3.         -0.4280246   1.93085389]
 [ 1.          1.          2.          0.52341555  0.05204635]
 [ 1.          1.          5.         -0.03047045  3.18499238]]
Omega:
 [0.26315294814541856, 0.7107986380800317]
Theta:
 [[-0.19315022  0.0421275 ]
 [-0.05870894 -0.27403636]]
Phi:
 [[0.71473496 0.01467602]
 [0.0330509  0.59746241]]


In [10]:
A = -Theta
print(f"A = {A}")
B = Phi+Theta
print(f"B = {B}")

A = [[ 0.19315022 -0.0421275 ]
 [ 0.05870894  0.27403636]]
B = [[ 0.52158474  0.05680351]
 [-0.02565804  0.32342605]]
